In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=46c866062672b4995a77f924dd9e251b72d031afae5c1ab605242818187639d2
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
pip install fpgrowth_py

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# Essential libraries
import pandas as pd
import numpy as np
import random
import time
from statistics import mean, variance, median, stdev
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as shc

# Scikit-learn libraries
from sklearn import datasets
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Keras libraries (if deep learning is used)
from keras.models import Model
from keras.layers import Dense, Input
from keras.preprocessing import sequence

# Loading dataset (example: Iris dataset)
from sklearn.datasets import load_iris

# Initialize scaler
scaler = StandardScaler()

In [ ]:
dataset_to_use = pd.read_csv('mtd_datasets/StaffAssign18.csv', index_col=False, dtype='unicode')
dataset_to_use

,AcademicYear,RecID,DistrictCode,Schoolcode,CountyName,DistrictName,SchoolName,StaffType,AssignmentCode,ClassID,CourseCode,EstimatedFTE,FileCreated
0,1819,1321406,5673940,6102230,VENTURA,Moorpark Unified,Chaparral Middle,T,NaN,03-01-Gym,2517,17.14,8/22/2019
1,1819,1350131,5673940,5633250,VENTURA,Moorpark Unified,Moorpark High,T,NaN,02-01-L-4,2798,20,8/22/2019
2,1819,1333164,5673940,6112189,VENTURA,Moorpark Unified,Mesa Verde Middle,T,NaN,02-01-R-3,2734,20,8/22/2019
3,1819,1299499,5673940,5633250,VENTURA,Moorpark Unified,Moorpark High,T,NaN,03-01-N-3,2130,20,8/22/2019
4,1819,1299499,5673940,5633250,VENTURA,Moorpark Unified,Moorpark High,T,NaN,04-01-N-3,2130,20,8/22/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1819,1235511,3768411,111831,SAN DIEGO,Sweetwater Union High,Olympian High,T,6017,NaN,NaN,25,8/22/2019
1048571,1819,1193025,3768411,0,SAN DIEGO,Sweetwater Union High,District Office,T,6017,NaN,NaN,100,8/22/2019
1048572,1819,1151321,3768411,0,SAN DIEGO,Sweetwater Union High,District Office,T,6017,NaN,NaN,100,8/22/2019
1048573,1819,1188223,3768411,0,SAN DIEGO,Sweetwater Union High,District Office,P,211,NaN,NaN,100,8/22/2019


In [ ]:
dataset_to_use = dataset_to_use[['CountyName', 'DistrictName',
       'SchoolName', 'ClassID', 'CourseCode']]
dataset_to_use

,CountyName,DistrictName,SchoolName,ClassID,CourseCode
0,VENTURA,Moorpark Unified,Chaparral Middle,03-01-Gym,2517
1,VENTURA,Moorpark Unified,Moorpark High,02-01-L-4,2798
2,VENTURA,Moorpark Unified,Mesa Verde Middle,02-01-R-3,2734
3,VENTURA,Moorpark Unified,Moorpark High,03-01-N-3,2130
4,VENTURA,Moorpark Unified,Moorpark High,04-01-N-3,2130
...,...,...,...,...,...
1048570,SAN DIEGO,Sweetwater Union High,Olympian High,NaN,NaN
1048571,SAN DIEGO,Sweetwater Union High,District Office,NaN,NaN
1048572,SAN DIEGO,Sweetwater Union High,District Office,NaN,NaN
1048573,SAN DIEGO,Sweetwater Union High,District Office,NaN,NaN


Index(['CountyName', 'DistrictName', 'SchoolName', 'ClassID', 'CourseCode'], dtype='object')

In [ ]:
dataset_to_use = dataset_to_use.iloc[1:40000, :]

,CountyName,DistrictName,SchoolName,ClassID,CourseCode
1,VENTURA,Moorpark Unified,Moorpark High,02-01-L-4,2798
2,VENTURA,Moorpark Unified,Mesa Verde Middle,02-01-R-3,2734
3,VENTURA,Moorpark Unified,Moorpark High,03-01-N-3,2130
4,VENTURA,Moorpark Unified,Moorpark High,04-01-N-3,2130
5,VENTURA,Moorpark Unified,Moorpark High,04-01-P-2,2414
...,...,...,...,...,...
39995,FRESNO,Fresno Unified,Sierra Charter,74,2535
39996,FRESNO,Fresno Unified,Sierra Charter,57,2403
39997,FRESNO,Fresno Unified,University High,432ACE-FH2,2515
39998,CONTRA COSTA,Antioch Unified,Antioch Middle,420_Y_4_6,6098


<ipython-input-27-636f685fad5e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_to_use[item] = dataset_to_use[item].astype('category')
<ipython-input-27-636f685fad5e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_to_use[item] = dataset_to_use[item].astype('category')
<ipython-input-27-636f685fad5e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,CountyName,DistrictName,SchoolName,ClassID,CourseCode
1,27,53,1341,463,252
2,27,53,1293,486,234
3,27,53,1341,906,18
4,27,53,1341,1421,18
5,27,53,1341,1429,95
...,...,...,...,...,...
39995,8,35,1777,26565,162
39996,8,35,1777,22976,88
39997,8,35,1936,20574,157
39998,6,8,135,20434,299


In [ ]:
import warnings
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Suppress specific DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

columns = ['CountyName', 'DistrictName', 'ClassID']

# Step 1: Focus on 'districtwide' and 'countyname' columns
df_filtered = dataset_to_use[columns]

# Step 1: Convert each row to a transaction format
transactions = df_filtered.apply(lambda x: list(x.dropna().astype(str)), axis=1).tolist()

# Step 2: One-hot encode the transactions
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
transaction_df = pd.DataFrame(te_ary, columns=te.columns_)

# Step 3: Apply FP-Growth to find frequent itemsets
frequent_itemsets = fpgrowth(transaction_df, min_support=0.0001, use_colnames=True)

# Step 4: Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0001)

# Step 5: Create a dictionary to track correlated attribute sets for each column
correlated_attributes = {col: set() for col in columns}

# Step 6: Populate correlated attributes for each column
for _, row in rules.iterrows():
    antecedents = list(row['antecedents'])
    consequents = list(row['consequents'])

    # Check if antecedents and consequents are from the same column
    for col in columns:
        col_antecedents = [item for item in antecedents if item in dataset_to_use[col].astype(str).unique()]
        col_consequents = [item for item in consequents if item in dataset_to_use[col].astype(str).unique()]

        # If the antecedents and consequents are in the same column, consider them correlated
        if col_antecedents and col_consequents:
            correlated_attributes[col].update(col_antecedents + col_consequents)

# Step 7: Output correlated attribute sets for each column
for col, attr_set in correlated_attributes.items():
    print(f"Correlated attributes for {col}: {attr_set}")


Correlated attributes for CountyName: set()
Correlated attributes for DistrictName: {'San Diego Unified', 'Lafayette Elementary', 'San Francisco Unified'}
Correlated attributes for SchoolName: {'Romoland Elementary', 'M. H. Stanley Middle', 'West Park Charter Academy', 'Bret Harte Union High', 'Harvest Valley Elementary', 'District Office', 'Ethan A Chase Middle', 'Boulder Ridge Elementary', 'John Vierra High', 'Lafayette Elementary', 'Mesa View Elementary', 'West Park Elementary'}
Correlated attributes for ClassID: {'9656711130', '02-01-Gym', '03-01-GYM', '07-01-Gym', '593623304', '3356757484', '05-01-Gym', '6001', '04-01-Gym', '32272591121', '532159', '2709', '2703', '67996441132', '271966303', '2207', '39251872', '96876009', '2613', '2517', '72995111', '2711', '3353-01', '1E+14', '4542752846', '699111584', '81861183', '2410', '03-01-Gym', '167781', '8465436302', '1851302', '06-01-Gym', '15789441129', '2626', '1186631186', '39172891128', '35216455739', '2.16583E+11', '2403', '2110', 